In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [96]:
import torch
from transformers import BertTokenizer, BertModel
from functools import partial
tokenizer = BertTokenizer.from_pretrained("deepset/bert-base-german-cased-hatespeech-GermEval18Coarse")
#model = BertModel.from_pretrained("deepset/bert-base-german-cased-hatespeech-GermEval18Coarse")

In [38]:
gt_text = """Zweck der Gesellschaft ist der Betrieb einer Bank. Ihr Geschäftskreis umfasst alle Arten von Bank-, Finanz-, Beratungs-, Dienstleistungs- und Handelsgeschäften im In- und Ausland. Die Gesellschaft kann im In- und Ausland Zweigniederlassungen und Vertretungen errichten sowie Banken, Finanzgesellschaften und andere Unternehmen aller Art gründen, sich an solchen beteiligen und deren Geschäftsführung übernehmen. Die Gesellschaft ist berechtigt, Grundstücke und Baurechte im In- und Ausland zu erwerben, zu belasten und zu verkaufen. Die Gesellschaft kann Mittel am Kapitalmarkt aufnehmen und anlegen. Die Gesellschaft ist Teil des Konzerns, der von UBS Group AG als Muttergesellschaft kontrolliert wird. Sie kann die Interessen der Konzernmuttergesellschaft oder anderer Konzerngesellschaften fördern. Sie kann Darlehen, Garantien und andere Arten der Finanzierung und von Sicherheitsleistungen für Konzerngesellschaften gewähren"""

In [235]:
md_zweck_text_original = """1 Zweck der Gesellschaft ist der Betrieb einer Bank. Ihr
Geschäftskreis umfasst alle Arten von Bank, Finanz-, Bera-
tungs-, Dienstleistungs- und Handelsgeschäften im In- und
Ausland.
2 Die Gesellschaft kann im In- und Ausland Zweigniederlas-
sungen und Vertretungen errichten sowie Banken, Finanz-
gesellschaften und andere Unternehmen aller Art gründen,
sich an solchen beteiligen und deren Geschäftsführung
übernehmen,
3 Die Gesellschaft ist berechtigt, Grundstücke und Baurechte
im In- und Ausland zu erwerben, zu belasten und zu ver-
kaufen.
4 Die Gesellschaft kann Mittel am Kapitalmarkt aufnehmen
und anlegen.
5 Die Gesellschaft ist Teil des Konzerns, der von UBS Group
AG als Muttergesellschaft kontrolliert wird. Sie kann die
Interessen der Konzernmuttergesellschaft oder anderer
Konzerngesellschaften fördern. Sie kann Darlehen, Garan-
tien und andere Arten der Finanzierung und von Sicher-
heitsleistungen für Konzerngesellschaften gewähren.
"""

In [40]:
md_NO_zweck_text = """Die Gesellschaft anerkennt nur einen Vertreter pro Aktie.

2 Das Stimmrecht und die damit zusammenhängenden

Rechte können der Gesellschaft gegenüber nur von einer

Person ausgeübt werden, die mit Stimmrecht im Aktien-

register eingetragen ist.

Abschnitt 3

Gesellschaftsorgane

A. Generalversammlung
## Zuständigkeit
"""

In [210]:
md_NO_zweck_text_2 = """Verwaltungsräte Artikel 18
Der Verwaltungsrat besteht aus mindestens fünf und
maximal zwölf Mitgliedern.


Amtsdauer"""

In [241]:
partial_encoder = partial(tokenizer.encode,return_tensors="pt",max_length=200, pad_to_max_length=True)


In [310]:
def textCleaner(text):
    text = text.replace("-\n","").replace("\n"," ")
    return ''.join([i for i in text if i.isalpha() or i == " "]).replace("  "," ").strip()

In [311]:
md_zweck_text_cleaned =  textCleaner(md_zweck_text)
print(md_zweck_text_cleaned)
print(textCleaner(gt_text))

Zweck der Gesellschaft ist der Betrieb einer Bank Ihr Geschäftskreis umfasst alle Arten von Bank Finanz Beratungs Dienstleistungs und Handelsgeschäften im In und Ausland Die Gesellschaft kann im In und Ausland Zweigniederlassungen und Vertretungen errichten sowie Banken Finanzgesellschaften und andere Unternehmen aller Art gründen sich an solchen beteiligen und deren Geschäftsführung übernehmen Die Gesellschaft ist berechtigt Grundstücke und Baurechte im In und Ausland zu erwerben zu belasten und zu verkaufen Die Gesellschaft kann Mittel am Kapitalmarkt aufnehmen und anlegen Die Gesellschaft ist Teil des Konzerns der von UBS Group AG als Muttergesellschaft kontrolliert wird Sie kann die Interessen der Konzernmuttergesellschaft oder anderer Konzerngesellschaften fördern Sie kann Darlehen Garantien und andere Arten der Finanzierung und von Sicherheitsleistungen für Konzerngesellschaften gewähren
Zweck der Gesellschaft ist der Betrieb einer Bank Ihr Geschäftskreis umfasst alle Arten von B

In [312]:
gt_token_encoded = partial_encoder(textCleaner(gt_text))
md_zweck_token_encoded = partial_encoder(textCleaner(md_zweck_text))
md_NO_zweck_token_encoded = partial_encoder(textCleaner(md_NO_zweck_text))
md_NO_zweck_2_encoded = partial_encoder(md_NO_zweck_text_2)

In [313]:
def calcCosSimBetweenTokens(token1, token2):
    return torch.nn.CosineSimilarity(dim=1, eps=1e-6)(token1,token2)

In [314]:
#print(calcCosSimBetweenTokens(gt_token_encoded,md_zweck_token_encoded))
#print(calcCosSimBetweenTokens(gt_token_encoded,md_NO_zweck_token_encoded))

In [315]:
from sklearn.metrics.pairwise import cosine_similarity

In [316]:
cosine_similarity(gt_token_encoded,md_zweck_token_encoded)[0][0]

1.0

In [317]:
cosine_similarity(gt_token_encoded,md_NO_zweck_token_encoded)[0][0]

0.14189327697392828

In [318]:
cosine_similarity(gt_token_encoded,md_NO_zweck_2_encoded)[0][0]

0.06043280376039879

In [319]:
cosine_similarity(gt_token_encoded,gt_token_encoded)[0][0]

1.0

In [296]:
t="ä"

In [297]:
t.isalpha()

True